# КОММЕНТАРИЙ РЕВЬЮВЕРА
Привет, Олег👋!)

Меня зовут Обидин Андрей, я осуществлял проверку проекта. 

Благодарю тебя за то, что вовремя всё сдал и хорошо поработал!)

`Олег, ты классно выполнил проект, у меня нет к тебе замечаний! Нравятся твои рассуждения и модельки, которые ты получил. Я оставил небольшие дополнения, думаю тебе будет интересно изучить их. Желаю успехов в дальнейшем обучении, так держать!`

Далее по ходу работы ты сможешь встретить мои комментарии, они будут отмечены разными цветами:

<div class="alert alert-block alert-success">
Похвала и возможности развития
</div>
<div class="alert alert-block alert-warning">
Некритичные замечания, но стоит обратить внимание
</div>
<div class="alert alert-block alert-danger">
Грубая ошибка, необходимо исправить
</div>


Заранее прошу не удалять мои комментарии, это важно. Если у тебя возникнут вопросы, обозначь их, я с радостью тебе помогу!)

Как вариант, можешь обозначить их так:
<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА:</b>
</div>

|  |  |  |
| ---: | :--- | :--- |
| Курс:| Введение в машинное обучение | 05 |
| Срок обучения на момент сдачи: | 2,5 месяца |

# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<a name="plan"></a>
## План:  
#### 1. [Изучение общей информации. Импорт библиотек, определение проблем.](#s1)  
#### 2. [Подготовка данных.](#s2)  
#### 3. [Исследование моделей.](#s3)  
*  [Решающее дерево.](#s3)  
*  [Случайный лес.](#s3_2)  
*  [Логистическая регрессия.](#s3_3)  
*  [Выбор модели.](#s3_4)  

#### 4. [Проверка модели на тестовой выборке.](#s4)    

#### 5. [Проверка модели на адекватность.](#s5)  
#### 6. [Заключение.](#s6)  

<a name="s1"></a>
## 1. Изучение общей информации. Импорт библиотек, определение проблем.

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from scipy import stats as st
from IPython.display import display
#import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [12]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [13]:
data.info()
display(data.head(4))
print(data['is_ultra'].value_counts())
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1


0    2229
1     985
Name: is_ultra, dtype: int64


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Данные без пропусков. Признаки: calls, minutes,	messages, mb_used. Целевой признак: is_ultra. 30% клиентов перешли на ultra.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Молодец, обзор хороший)
    
Классно, что ты решил изучить распределение классов, это полезно, так как помогает детектировать проблему дисбаланса, которая негативно сказывается на моделях. Как справляться с этой проблемой ты узнаешь в следующем курсе)

</div>

[вернуться к оглавлению](#plan) 
<a name="s2"></a>
## 2. Подготовка данных.
Разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [14]:
data_train, data_testvalid = train_test_split(data, test_size=0.4, random_state=123)
data_test, data_valid = train_test_split(data_testvalid, test_size=0.5, random_state=123)

In [15]:
display(data_train.describe())
display(data_valid.describe())
display(data_test.describe()) 

,calls,minutes,messages,mb_used,is_ultra
count,1928.000000,1928.000000,1928.000000,1928.000000,1928.000000
mean,63.117220,439.033734,38.156120,17203.078242,0.308091
std,33.439266,236.342509,36.684478,7556.799702,0.461824
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,276.607500,9.000000,12472.822500,0.000000
50%,62.000000,430.090000,29.000000,16849.685000,0.000000
75%,82.000000,578.700000,56.000000,21326.615000,1.000000
max,203.000000,1566.450000,224.000000,48351.610000,1.000000


,calls,minutes,messages,mb_used,is_ultra
count,643.000000,643.000000,643.000000,643.000000,643.000000
mean,63.570762,442.875801,37.388802,17156.622877,0.306376
std,32.937388,230.857527,34.698990,7428.841067,0.461347
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,281.270000,8.000000,12706.160000,0.000000
50%,63.000000,447.070000,30.000000,16917.440000,0.000000
75%,82.500000,571.150000,58.500000,21584.375000,1.000000
max,244.000000,1632.060000,188.000000,43931.390000,1.000000


,calls,minutes,messages,mb_used,is_ultra
count,643.000000,643.000000,643.000000,643.000000,643.000000
mean,62.272162,431.068212,39.548989,17272.504432,0.301711
std,32.960360,233.113206,35.969293,7763.148353,0.459358
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,38.000000,263.295000,10.000000,12453.180000,0.000000
50%,60.000000,419.720000,31.000000,17275.470000,0.000000
75%,83.000000,557.580000,57.500000,21515.760000,1.000000
max,189.000000,1364.830000,201.000000,49745.730000,1.000000


Целевой признак в обучающую, валидационную и тестовую выборки распределился равномерно.

In [16]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Отлично, что ты не забыл разбить датафрейм именно на три выборки!

</div>

[вернуться к оглавлению](#plan) 
<a name="s3"></a>
## 3. Исследование моделей.
Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

<a name="s3"></a>
### Решающее дерево.

In [17]:
for depth in range (1, 5):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth =", depth, ", accuracy_score =", accuracy_score(target_valid, predictions_valid))

max_depth = 1 , accuracy_score = 0.7558320373250389
max_depth = 2 , accuracy_score = 0.7900466562986003
max_depth = 3 , accuracy_score = 0.7962674961119751
max_depth = 4 , accuracy_score = 0.7838258164852255


Наилучший результат на валидационной выборке для решающего дерева с глубиной 3.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Ради интереса я бы взял чуть больший диапазон глубины, так как в пределах 2-4 доля правильных ответов существенно не меняется, давай посмотим, мало ли?

</div>

In [18]:
# КОД РЕВЬЮВЕРА

rev_tree = None
rev_tree_depth = 0
accuracy_best = 0.5 # психологически порог случайной модели, который наше дерево в любом случае должно преодолеть, чтобы мы его не срубили с корнями))

for depth in range(2, 15): 
    
    tree = DecisionTreeClassifier(random_state = 500, max_depth = depth)
    tree.fit(features_train, target_train)
    
    acc = tree.score(features_valid, target_valid)
    
    if acc > accuracy_best:
        
        accuracy_best = acc
        rev_tree = tree
        rev_tree_depth = depth
        
print('Дерево ревьювера')
print('-------> глубина', rev_tree_depth)
print('-------> accuracy', accuracy_best)


Дерево ревьювера
-------> глубина 3
-------> accuracy 0.7962674961119751


<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Глубина три в данном случае судя по всему является оптимальной

</div>

<a name="s3_2"></a> 
### Случайный лес.

In [24]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 17):
    for depth in range(1, 5):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_RandomForest_model = model
            best_est = est
            best_result = result
            best_depth = depth

print("Accuracy лучшей модели:", best_result, "для леса из", best_est, "деревьев глубиной", best_depth, ".")

Accuracy лучшей модели: 0.807153965785381 для леса из 12 деревьев глубиной 2 .


Качество у случайного леса (80.7%) не сильно выше чем для решающего дерева (80.6%).

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Лес действительно дает большее качество, но и работает дольше

</div>

<a name="s3_3"></a> 
### Логистическая регрессия.

In [25]:
model = LogisticRegression(random_state=123, solver='liblinear')
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии:", result)

Accuracy модели логистической регрессии: 0.7620528771384136


<a name="s3_4"></a> 
### Выбор модели.

Хуже всего на валидационной выборке себя показала логистическая регрессия (76,2%). Результаты для решающего дерева и случайного леса практически одинаковые (80,6% и 80,7% соответственно). Для поставленных целей разница в точности в 0,1% не будет являеться решающей. В качестве модели можно выбрать решающее дерево глубиной 3.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Да, регрессия действительно работает плоховато, а вот если говорить про лес \ дерево, то они явно конкурируют и я бы тоже выбрал дерево, так как эта модель является балансом между скоростью работы и качеством

</div>

[вернуться к оглавлению](#plan) 
<a name="s4"></a>
## 4. Проверка модели на тестовой выборке.
Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

In [40]:
model = DecisionTreeClassifier(random_state=123, max_depth=3)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
print("Точность на тестовой выборке:", accuracy_score(target_test, predictions_test))

Точность на тестовой выборке: 0.7589424572317263


<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Видим, что порог мы преодолели, но давай посмотрим, как покажет себя лес?

</div>

In [19]:
# КОД РЕВЬЮВЕРА

forest = RandomForestClassifier(random_state = 500, n_estimators = 12, max_depth = 2)
forest.fit(features_train, target_train)

print(forest.score(features_test, target_test))

0.7807153965785381


<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
Работает лучше, чем дерево

</div>

[вернуться к оглавлению](#plan) 
<a name="s5"></a>
## 5. (бонус) Проверка модели на адекватность.

Очевидно, что заполнив предсказание нулями получим точность 70%. 
Первым делом была мысль попробовать создать предсказание по собственному алгоритму (например заполнить единицами 30% наиболее активных абонентов) и посмотреть результат, но не уверен что это то, что подразумевалось в этом задании.  
Попробуем псевдопредсказатель DummyClassifier. 
  'most_frequent' - заполнит предсказание наиболее распространенным значением 
  'stratified' - заполнит значениями с сохранением доли в тестовой выборке
  'uniform' - заполнит значениями в равной пропорции

In [43]:
strategies = ['most_frequent', 'stratified', 'uniform']
for s in strategies:
    dclf = DummyClassifier(strategy = s, random_state = 123) 
    dclf.fit(features_train, target_train)
    result = dclf.score(features_valid, target_valid)
    print(s, result)

most_frequent 0.7091757387247278
stratified 0.5925349922239502
uniform 0.46500777604976673


[вернуться к оглавлению](#plan) 
<a name="s6"></a>
## 5. Заключение.

Модель решающее дерево глубиной 3 дает точность более 75%. Это на 5% лучше чем во всех случаях предсказывать наиболее популярный тариф. Для поставленной задачи такой точности достаточно, но для более серьезных целей необходимо увеличивать точность.

<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
У тебя довольно разумная проверка на адекватность, молодец!

Хочу обратить твое внимание на регрессию, как ты помнишь доля правильных ответов по ней на валидации была 0.76, а у модели, которая предсказывает только наиболее популярный класс 0.7, недалеко уехали.
    
Хочу еще рассказать тебе о метрике auc, о ней ты узнаешь уже буквально в следующих курсах. На данный момент скажу, что auc случайной модели равен 0.5, чем выше и дальше auc  нашей модели от этого показателя, тем лучше.
    
Подсчитаем:

</div>

In [21]:
# КОД РЕВЬЮВЕРА

# для интереса я возьму тот лес, что обучил выше

from sklearn.metrics import roc_auc_score

# для подсчета auc нам нужны не предсказания, а вероятность принадлежность объекта к классу

prob = forest.predict_proba(features_test)[:, 1]

print('AUC', roc_auc_score(target_test, prob))

# ВЫВОД: видим, что метрика куда выше 0.5, делаем вывод, что модель леса работает вполне адекватно

AUC 0.7397423828438914


<div class="alert alert-block alert-success">
<b>КОММЕНТАРИЙ РЕВЬЮВЕРА:</b> 
    
    
`Олег, ты классно выполнил проект, у меня нет к тебе замечаний, молодец! Ради подогревания твоего интереса скажу, как можно было еще улучшить модели: 1. У нас есть проблема дисбаланса классов(можно было установить параметр class_weight = 'balanced' \ провести upsampling (увеличение объектов меньшего класса) \ downsampling (уменьшить кол-во объектов большего класса) 2. Провести масштабирование признаков. Обо всем этом ты узнаешь уже в следующем курсе, желаю удачи!`

</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
